# google-covid-19-mobility-data-process-india

In [54]:
import pandas as pd
import requests
import simplejson as json
import numpy as np

In [2]:
indiaDf = pd.read_csv("./output-data/india.csv")

In [3]:
indiaDf.head()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,IN,India,NaN,NaN,NaN,NaN,NaN,ChIJkbeSa_BfYzARphNChaFPjNc,2020-02-15,1.0,2.0,3.0,3.0,5.0,0.0
1,IN,India,NaN,NaN,NaN,NaN,NaN,ChIJkbeSa_BfYzARphNChaFPjNc,2020-02-16,2.0,2.0,3.0,2.0,0.0,0.0
2,IN,India,NaN,NaN,NaN,NaN,NaN,ChIJkbeSa_BfYzARphNChaFPjNc,2020-02-17,-1.0,1.0,3.0,1.0,4.0,0.0
3,IN,India,NaN,NaN,NaN,NaN,NaN,ChIJkbeSa_BfYzARphNChaFPjNc,2020-02-18,0.0,2.0,4.0,2.0,3.0,0.0
4,IN,India,NaN,NaN,NaN,NaN,NaN,ChIJkbeSa_BfYzARphNChaFPjNc,2020-02-19,0.0,2.0,1.0,1.0,4.0,1.0


In [4]:
len(indiaDf)

307294

In [7]:
indiaDf["sub_region_1"].unique()

array([nan, 'Andaman and Nicobar Islands', 'Andhra Pradesh',
       'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh',
       'Chhattisgarh', 'Dadra and Nagar Haveli', 'Daman and Diu', 'Delhi',
       'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh',
       'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala',
       'Lakshadweep', 'Madhya Pradesh', 'Maharashtra', 'Manipur',
       'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Puducherry',
       'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana',
       'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal'],
      dtype=object)

In [8]:
indiaDf["sub_region_2"].unique()

array([nan, 'North and Middle Andaman', 'South Andaman', 'Anantapuram',
       'Chittoor', 'East Godavari', 'Guntur', 'Krishna', 'Kurnool',
       'Prakasam', 'Sri Potti Sriramulu Nellore District', 'Srikakulam',
       'Vishakhapatnam', 'Vizianagaram', 'West Godavari', 'YSR District',
       'East Kameng', 'East Siang', 'Papum Pare', 'Tawang', 'West Kameng',
       'Baksa', 'Barpeta', 'Bongaigaon', 'Cachar', 'Chirang', 'Darrang',
       'Dhemaji', 'Dhubri', 'Dibrugarh', 'Dima Hasao', 'Goalpara',
       'Golaghat', 'Hailakandi', 'Jorhat', 'Kamrup',
       'Kamrup Metropolitan', 'Karbi Anglong', 'Karimganj', 'Kokrajhar',
       'Lakhimpur', 'Morigaon', 'Nagaon', 'Nalbari', 'Sivasagar',
       'Sonitpur', 'Tinsukia', 'Udalguri', 'Araria', 'Arwal',
       'Aurangabad', 'Banka', 'Begusarai', 'Bhagalpur', 'Bhojpur',
       'Buxar', 'Darbhanga', 'Gaya', 'Gopalganj', 'Jamui', 'Jehanabad',
       'Kaimur', 'Katihar', 'Khagaria', 'Kishanganj', 'Lakhisarai',
       'Madhepura', 'Madhubani', 'Mun

In [9]:
len(indiaDf["sub_region_2"].unique())

628

In [11]:
indiaDf["metro_area"].unique()

array([nan])

---

## Create a new data frame containing just the rows with `sub_region_2`

In [12]:
indiaSubRegion2Df = indiaDf[indiaDf["sub_region_2"].notnull()]

In [13]:
len(indiaSubRegion2Df)

290228

## Google Maps

In [14]:
uniquePlaceIdsDf = indiaSubRegion2Df[["place_id"]].drop_duplicates()

In [15]:
len(uniquePlaceIdsDf)

633

In [16]:
with open('./secrets/googleapikey.txt', 'r') as f:
    key = f.read()

In [17]:
def get_lat_long(place_id):
    try:
        API_KEY = key.rstrip("\n")
        url = "https://maps.googleapis.com/maps/api/place/details/json?place_id=" + str(place_id) + "&key=" + str(API_KEY) + "&fields=geometry"
        
        response = (requests.get(url).text)
        response_json = json.loads(response)
        
        if "result" in response_json:
            result = response_json["result"]
            if "geometry" in result:
                geometry = result["geometry"]
                if "location" in geometry:
                    location = geometry["location"]
                    return location["lat"], location["lng"]
                else:
                    return None, None
            else:
                return None, None
        else:
            return None, None
    
    except Exception as e:
        raise e

In [18]:
uniquePlaceIdsDf.loc[:, "lat"], uniquePlaceIdsDf.loc[:, "lng"] = zip(*uniquePlaceIdsDf['place_id'].map(get_lat_long))

In [19]:
uniquePlaceIdsDf.head()

,place_id,lat,lng
921,ChIJs5lpAH2kjzARilD7p7n6fns,12.651883,92.897625
1246,ChIJEU2QUeUOjzAR9ynzlt7zsYk,11.740087,92.658640
2170,ChIJ7UI4G8ZKsTsRi2DdoCb6SF4,14.682378,77.601656
2632,ChIJWcp9dxxcrTsRCqBG4peLZeI,13.478791,78.838264
3094,ChIJ97RZr90nODoRK6G-vlm6C2I,17.321250,82.040714


In [22]:
uniquePlaceIdsDf.to_csv("./output-data/india-sub-region-2-lat-lng.csv", index=False)

In [21]:
len(uniquePlaceIdsDf)

633

---

## Merge the coordinates with the original `sub_region_2` data frame

In [23]:
indiaSubRegion2MergeDf = pd.merge(indiaSubRegion2Df, uniquePlaceIdsDf, on='place_id', how='outer')

In [24]:
indiaSubRegion2MergeDf.head()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,lat,lng
0,IN,India,Andaman and Nicobar Islands,North and Middle Andaman,NaN,NaN,NaN,ChIJs5lpAH2kjzARilD7p7n6fns,2020-03-01,NaN,NaN,-41.0,NaN,NaN,NaN,12.651883,92.897625
1,IN,India,Andaman and Nicobar Islands,North and Middle Andaman,NaN,NaN,NaN,ChIJs5lpAH2kjzARilD7p7n6fns,2020-03-08,NaN,NaN,-53.0,NaN,NaN,NaN,12.651883,92.897625
2,IN,India,Andaman and Nicobar Islands,North and Middle Andaman,NaN,NaN,NaN,ChIJs5lpAH2kjzARilD7p7n6fns,2020-03-09,NaN,NaN,NaN,NaN,-43.0,NaN,12.651883,92.897625
3,IN,India,Andaman and Nicobar Islands,North and Middle Andaman,NaN,NaN,NaN,ChIJs5lpAH2kjzARilD7p7n6fns,2020-03-10,NaN,NaN,NaN,NaN,-51.0,NaN,12.651883,92.897625
4,IN,India,Andaman and Nicobar Islands,North and Middle Andaman,NaN,NaN,NaN,ChIJs5lpAH2kjzARilD7p7n6fns,2020-03-15,NaN,NaN,-40.0,NaN,NaN,NaN,12.651883,92.897625


---

## Calculate 7 day rolling average

In [25]:
def add_rolling_average(df):
    df.loc[:, "retail-average"] = df["retail_and_recreation_percent_change_from_baseline"].rolling(window=7, center=True).mean()
    df.loc[:, "grocery-average"] = df["grocery_and_pharmacy_percent_change_from_baseline"].rolling(window=7, center=True).mean()
    df.loc[:, "parks-average"] = df["parks_percent_change_from_baseline"].rolling(window=7, center=True).mean()
    df.loc[:, "transit-average"] = df["transit_stations_percent_change_from_baseline"].rolling(window=7, center=True).mean()
    df.loc[:, "workplace-average"] = df["workplaces_percent_change_from_baseline"].rolling(window=7, center=True).mean()
    df.loc[:, "residential-average"] = df["residential_percent_change_from_baseline"].rolling(window=7, center=True).mean()
    return df

In [26]:
indiaSubRegion2AverageDf = indiaSubRegion2MergeDf.groupby("place_id").apply(add_rolling_average)

In [28]:
indiaSubRegion2AverageDf[500:505]

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,...,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,lat,lng,retail-average,grocery-average,parks-average,transit-average,workplace-average,residential-average
500,IN,India,Andaman and Nicobar Islands,South Andaman,NaN,NaN,NaN,ChIJEU2QUeUOjzAR9ynzlt7zsYk,2020-08-08,-93.0,...,-67.0,39.0,11.740087,92.65864,-77.571429,NaN,-77.285714,-53.428571,-50.714286,NaN
501,IN,India,Andaman and Nicobar Islands,South Andaman,NaN,NaN,NaN,ChIJEU2QUeUOjzAR9ynzlt7zsYk,2020-08-09,-88.0,...,-42.0,NaN,11.740087,92.65864,-77.000000,NaN,-76.428571,-50.714286,-50.428571,NaN
502,IN,India,Andaman and Nicobar Islands,South Andaman,NaN,NaN,NaN,ChIJEU2QUeUOjzAR9ynzlt7zsYk,2020-08-10,-63.0,...,-42.0,29.0,11.740087,92.65864,-76.428571,NaN,-75.714286,-48.428571,-49.857143,NaN
503,IN,India,Andaman and Nicobar Islands,South Andaman,NaN,NaN,NaN,ChIJEU2QUeUOjzAR9ynzlt7zsYk,2020-08-11,-67.0,...,-46.0,32.0,11.740087,92.65864,-74.714286,NaN,-74.571429,-45.857143,-48.428571,NaN
504,IN,India,Andaman and Nicobar Islands,South Andaman,NaN,NaN,NaN,ChIJEU2QUeUOjzAR9ynzlt7zsYk,2020-08-12,-74.0,...,-50.0,34.0,11.740087,92.65864,-74.000000,NaN,-73.571429,-43.857143,-47.142857,NaN


---

## Round the average figures to 1 decimal place for a smaller final file size

In [39]:
indiaSubRegion2RoundedDf = indiaSubRegion2AverageDf.round({
    'retail-average': 0,
    'grocery-average': 0,
    'parks-average': 0,
    'transit-average': 0,
    'workplace-average': 0,
    'residential-average': 0,
})

In [40]:
indiaSubRegion2RoundedDf.tail()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,...,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,lat,lng,retail-average,grocery-average,parks-average,transit-average,workplace-average,residential-average
290223,IN,India,West Bengal,West Medinipur,NaN,NaN,NaN,ChIJJXVMlY9CHToR4mlOVIq_xDQ,2021-05-17,-54.0,...,-42.0,33.0,22.408038,87.381073,-48.0,-20.0,-12.0,-48.0,-35.0,29.0
290224,IN,India,West Bengal,West Medinipur,NaN,NaN,NaN,ChIJJXVMlY9CHToR4mlOVIq_xDQ,2021-05-18,-53.0,...,-43.0,31.0,22.408038,87.381073,-50.0,-23.0,-13.0,-50.0,-34.0,30.0
290225,IN,India,West Bengal,West Medinipur,NaN,NaN,NaN,ChIJJXVMlY9CHToR4mlOVIq_xDQ,2021-05-19,-53.0,...,-37.0,28.0,22.408038,87.381073,NaN,NaN,NaN,NaN,NaN,NaN
290226,IN,India,West Bengal,West Medinipur,NaN,NaN,NaN,ChIJJXVMlY9CHToR4mlOVIq_xDQ,2021-05-20,-48.0,...,-30.0,29.0,22.408038,87.381073,NaN,NaN,NaN,NaN,NaN,NaN
290227,IN,India,West Bengal,West Medinipur,NaN,NaN,NaN,ChIJJXVMlY9CHToR4mlOVIq_xDQ,2021-05-21,-50.0,...,-34.0,31.0,22.408038,87.381073,NaN,NaN,NaN,NaN,NaN,NaN


---

## Remove any NaN `place_id`s

In [41]:
indiaSubRegion2NotNaDf = indiaSubRegion2RoundedDf[indiaSubRegion2RoundedDf["place_id"].notna(
)]

---

## Convert the data into a python dictionary so it can be exported as json

In [34]:
dates = ["2020-02-15","2020-02-16","2020-02-17","2020-02-18","2020-02-19","2020-02-20","2020-02-21","2020-02-22","2020-02-23","2020-02-24","2020-02-25","2020-02-26","2020-02-27","2020-02-28","2020-02-29","2020-03-01","2020-03-02","2020-03-03","2020-03-04","2020-03-05","2020-03-06","2020-03-07","2020-03-08","2020-03-09","2020-03-10","2020-03-11","2020-03-12","2020-03-13","2020-03-14","2020-03-15","2020-03-16","2020-03-17","2020-03-18","2020-03-19","2020-03-20","2020-03-21","2020-03-22","2020-03-23","2020-03-24","2020-03-25","2020-03-26","2020-03-27","2020-03-28","2020-03-29","2020-03-30","2020-03-31","2020-04-01","2020-04-02","2020-04-03","2020-04-04","2020-04-05","2020-04-06","2020-04-07","2020-04-08","2020-04-09","2020-04-10","2020-04-11","2020-04-12","2020-04-13","2020-04-14","2020-04-15","2020-04-16","2020-04-17","2020-04-18","2020-04-19","2020-04-20","2020-04-21","2020-04-22","2020-04-23","2020-04-24","2020-04-25","2020-04-26","2020-04-27","2020-04-28","2020-04-29","2020-04-30","2020-05-01","2020-05-02","2020-05-03","2020-05-04","2020-05-05","2020-05-06","2020-05-07","2020-05-08","2020-05-09","2020-05-10","2020-05-11","2020-05-12","2020-05-13","2020-05-14","2020-05-15","2020-05-16","2020-05-17","2020-05-18","2020-05-19","2020-05-20","2020-05-21","2020-05-22","2020-05-23","2020-05-24","2020-05-25","2020-05-26","2020-05-27","2020-05-28","2020-05-29","2020-05-30","2020-05-31","2020-06-01","2020-06-02","2020-06-03","2020-06-04","2020-06-05","2020-06-06","2020-06-07","2020-06-08","2020-06-09","2020-06-10","2020-06-11","2020-06-12","2020-06-13","2020-06-14","2020-06-15","2020-06-16","2020-06-17","2020-06-18","2020-06-19","2020-06-20","2020-06-21","2020-06-22","2020-06-23","2020-06-24","2020-06-25","2020-06-26","2020-06-27","2020-06-28","2020-06-29","2020-06-30","2020-07-01","2020-07-02","2020-07-03","2020-07-04","2020-07-05","2020-07-06","2020-07-07","2020-07-08","2020-07-09","2020-07-10","2020-07-11","2020-07-12","2020-07-13","2020-07-14","2020-07-15","2020-07-16","2020-07-17","2020-07-18","2020-07-19","2020-07-20","2020-07-21","2020-07-22","2020-07-23","2020-07-24","2020-07-25","2020-07-26","2020-07-27","2020-07-28","2020-07-29","2020-07-30","2020-07-31","2020-08-01","2020-08-02","2020-08-03","2020-08-04","2020-08-05","2020-08-06","2020-08-07","2020-08-08","2020-08-09","2020-08-10","2020-08-11","2020-08-12","2020-08-13","2020-08-14","2020-08-15","2020-08-16","2020-08-17","2020-08-18","2020-08-19","2020-08-20","2020-08-21","2020-08-22","2020-08-23","2020-08-24","2020-08-25","2020-08-26","2020-08-27","2020-08-28","2020-08-29","2020-08-30","2020-08-31","2020-09-01","2020-09-02","2020-09-03","2020-09-04","2020-09-05","2020-09-06","2020-09-07","2020-09-08","2020-09-09","2020-09-10","2020-09-11","2020-09-12","2020-09-13","2020-09-14","2020-09-15","2020-09-16","2020-09-17","2020-09-18","2020-09-19","2020-09-20","2020-09-21","2020-09-22","2020-09-23","2020-09-24","2020-09-25","2020-09-26","2020-09-27","2020-09-28","2020-09-29","2020-09-30","2020-10-01","2020-10-02","2020-10-03","2020-10-04","2020-10-05","2020-10-06","2020-10-07","2020-10-08","2020-10-09","2020-10-10","2020-10-11","2020-10-12","2020-10-13","2020-10-14","2020-10-15","2020-10-16","2020-10-17","2020-10-18","2020-10-19","2020-10-20","2020-10-21","2020-10-22","2020-10-23","2020-10-24","2020-10-25","2020-10-26","2020-10-27","2020-10-28","2020-10-29","2020-10-30","2020-10-31","2020-11-01","2020-11-02","2020-11-03","2020-11-04","2020-11-05","2020-11-06","2020-11-07","2020-11-08","2020-11-09","2020-11-10","2020-11-11","2020-11-12","2020-11-13","2020-11-14","2020-11-15","2020-11-16","2020-11-17","2020-11-18","2020-11-19","2020-11-20","2020-11-21","2020-11-22","2020-11-23","2020-11-24","2020-11-25","2020-11-26","2020-11-27","2020-11-28","2020-11-29","2020-11-30","2020-12-01","2020-12-02","2020-12-03","2020-12-04","2020-12-05","2020-12-06","2020-12-07","2020-12-08","2020-12-09","2020-12-10","2020-12-11","2020-12-12","2020-12-13","2020-12-14","2020-12-15","2020-12-16","2020-12-17","2020-12-18","2020-12-19","2020-12-20","2020-12-21","2020-12-22","2020-12-23","2020-12-24","2020-12-25","2020-12-26","2020-12-27","2020-12-28","2020-12-29","2020-12-30","2020-12-31","2021-01-01","2021-01-02","2021-01-03","2021-01-04","2021-01-05","2021-01-06","2021-01-07","2021-01-08","2021-01-09","2021-01-10","2021-01-11","2021-01-12","2021-01-13","2021-01-14","2021-01-15","2021-01-16","2021-01-17","2021-01-18","2021-01-19","2021-01-20","2021-01-21","2021-01-22","2021-01-23","2021-01-24","2021-01-25","2021-01-26","2021-01-27","2021-01-28","2021-01-29","2021-01-30","2021-01-31","2021-02-01","2021-02-02","2021-02-03","2021-02-04","2021-02-05","2021-02-06","2021-02-07","2021-02-08","2021-02-09","2021-02-10","2021-02-11","2021-02-12","2021-02-13","2021-02-14","2021-02-15","2021-02-16","2021-02-17","2021-02-18","2021-02-19","2021-02-20","2021-02-21","2021-02-22","2021-02-23","2021-02-24","2021-02-25","2021-02-26","2021-02-27","2021-02-28","2021-03-01","2021-03-02","2021-03-03","2021-03-04","2021-03-05","2021-03-06","2021-03-07","2021-03-08","2021-03-09","2021-03-10","2021-03-11","2021-03-12","2021-03-13","2021-03-14","2021-03-15","2021-03-16","2021-03-17","2021-03-18","2021-03-19","2021-03-20","2021-03-21","2021-03-22","2021-03-23","2021-03-24","2021-03-25","2021-03-26","2021-03-27","2021-03-28","2021-03-29","2021-03-30","2021-03-31","2021-04-01","2021-04-02","2021-04-03","2021-04-04","2021-04-05","2021-04-06","2021-04-07","2021-04-08","2021-04-09","2021-04-10","2021-04-11","2021-04-12","2021-04-13","2021-04-14","2021-04-15","2021-04-16","2021-04-17","2021-04-18","2021-04-19","2021-04-20","2021-04-21","2021-04-22","2021-04-23","2021-04-24","2021-04-25","2021-04-26","2021-04-27","2021-04-28","2021-04-29","2021-04-30","2021-05-01","2021-05-02","2021-05-03","2021-05-04","2021-05-05","2021-05-06","2021-05-07","2021-05-08","2021-05-09","2021-05-10","2021-05-11","2021-05-12","2021-05-13","2021-05-14","2021-05-15","2021-05-16","2021-05-17","2021-05-18","2021-05-19","2021-05-20","2021-05-21"]

In [60]:
def create_list_for_json(df):
    outputList = []
    listOfPlaceIds = df["place_id"].drop_duplicates().to_list()
    groupByPlaceId = df.groupby("place_id")
    
    for place_id in listOfPlaceIds:
        thisDf = groupByPlaceId.get_group(place_id)
        testList = thisDf["parks-average"].to_list()
        
        # Some of the parks columns contain all NaNs, we'll skip these
        if np.isnan(testList).all():
            continue
        
        myDict = {}
        myDict['lng'] = thisDf.iloc[0]["lng"]
        myDict["lat"] =  thisDf.iloc[0]["lat"]

        dateIndexDf = thisDf.set_index("date")
        
        # RETAIL
        
        retailList = []
        
        for date in dates:
            thisDate = np.nan
            try:
                thisDate = int(dateIndexDf.loc[date]["retail-average"])
            except:
                thisDate = np.nan
            
            retailList.append(thisDate)
        
        myDict["retail"] = retailList
        
        # GROCERY
        
        groceryList = []
        
        for date in dates:
            thisDate = np.nan
            try:
                thisDate = int(dateIndexDf.loc[date]["grocery-average"])
            except:
                thisDate = np.nan
            
            groceryList.append(thisDate)
        
        myDict["grocery"] = groceryList
        
        # PARKS
        
        parksList = []
        
        for date in dates:
            thisDate = np.nan
            try:
                thisDate = int(dateIndexDf.loc[date]["parks-average"])
            except:
                thisDate = np.nan
            
            parksList.append(thisDate)
        
        myDict["parks"] = parksList
        
        # TRANSIT
        
        transitList = []
        
        for date in dates:
            thisDate = np.nan
            try:
                thisDate = int(dateIndexDf.loc[date]["transit-average"])
            except:
                thisDate = np.nan
            
            transitList.append(thisDate)
        
        myDict["transit"] = transitList
        
        # WORKPLACE
        
        workplaceList = []
        
        for date in dates:
            thisDate = np.nan
            try:
                thisDate = int(dateIndexDf.loc[date]["workplace-average"])
            except:
                thisDate = np.nan
            
            workplaceList.append(thisDate)
        
        myDict["workplace"] = workplaceList
        
        # Residential
        
        residentialList = []
        
        for date in dates:
            thisDate = np.nan
            try:
                thisDate = int(dateIndexDf.loc[date]["residential-average"])
            except:
                thisDate = np.nan
            
            residentialList.append(thisDate)
        
        myDict["residential"] = residentialList
        

        outputList.append(myDict)
        
    return outputList

In [61]:
indiaSubRegion2List = create_list_for_json(indiaSubRegion2NotNaDf)

In [62]:
indiaSubRegion2List[0]

{'lng': 92.6586401,
 'lat': 11.7400867,
 'retail': [nan,
  nan,
  nan,
  -3,
  -4,
  -4,
  -5,
  -5,
  -5,
  -5,
  -5,
  -5,
  -5,
  -5,
  -7,
  -7,
  -7,
  -8,
  -10,
  -11,
  -12,
  -15,
  -16,
  -17,
  -17,
  -18,
  -19,
  -21,
  -19,
  -20,
  -21,
  -26,
  -32,
  -39,
  -45,
  -51,
  -60,
  -69,
  -74,
  -76,
  -75,
  -76,
  -77,
  -76,
  -76,
  -75,
  -75,
  -77,
  -78,
  -79,
  -80,
  -81,
  -82,
  -83,
  -84,
  -85,
  -85,
  -85,
  -86,
  -86,
  -87,
  -87,
  -88,
  -88,
  -89,
  -89,
  -89,
  -90,
  -91,
  -91,
  -91,
  -91,
  -91,
  -91,
  -91,
  -90,
  -90,
  -90,
  -90,
  -89,
  -89,
  -88,
  -88,
  -88,
  -87,
  -87,
  -87,
  -87,
  -87,
  -87,
  -86,
  -86,
  -85,
  -84,
  -83,
  -82,
  -81,
  -81,
  -81,
  -80,
  -79,
  -79,
  -78,
  -78,
  -76,
  -75,
  -75,
  -74,
  -73,
  -73,
  -72,
  -72,
  -72,
  -71,
  -71,
  -70,
  -69,
  -68,
  -67,
  -66,
  -66,
  -66,
  -66,
  -65,
  -64,
  -64,
  -62,
  -61,
  -60,
  -59,
  -59,
  -59,
  -60,
  -59,
  -58,
  -57,
  -57,
  -55,

In [63]:
len(indiaSubRegion2List)

607

In [64]:
with open("./public/data/india-all.json", "w") as outfile: 
    json.dump(indiaSubRegion2List, outfile, ignore_nan=True)